In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf


In [ ]:
num_vectors = 1000
num_clusters = 2
num_steps = 100

2つのガウス分布からサンプル点を生成

In [ ]:
vector_values = []
for i in range(num_vectors):
  if np.random.random() > 0.5:
    vector_values.append([np.random.normal(0.5, 0.6),
                          np.random.normal(0.3, 0.9)])
  else:
    vector_values.append([np.random.normal(2.5, 0.4),
                         np.random.normal(0.8, 0.5)])

In [ ]:
df = pd.DataFrame({"x": [v[0] for v in vector_values], 
                   "y": [v[1] for v in vector_values]})
sns.lmplot("x", "y", data=df, fit_reg=False, size=7)
plt.show()

入力データを定数   
重心を適当な点とする

In [ ]:
vectors = tf.constant(vector_values)
centroids = tf.Variable(tf.slice(tf.random_shuffle(vectors), 
                                 [0,0],[num_clusters,-1]))

次元を整える？

In [ ]:
expanded_vectors = tf.expand_dims(vectors, 0)
expanded_centroids = tf.expand_dims(centroids, 1)

print(expanded_vectors.get_shape())
print(expanded_centroids.get_shape())

In [ ]:
distances = tf.reduce_sum(
  tf.square(tf.subtract(expanded_vectors, expanded_centroids)), 2)
assignments = tf.argmin(distances, 0)

In [ ]:
means = tf.stack([
    tf.reduce_mean(
        tf.boolean_mask(
            vectors, tf.equal(assignments, c)
        ), 0) 
    for c in range(num_clusters)])

In [ ]:
update_centroids  = tf.assign(centroids, means)
init_op = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
  sess.run(init_op)
  for step in range(num_steps):
    _, centroid_values, assignment_values = sess.run([update_centroids,centroids,assignments])
  print("centroids")
  print(centroid_values)

In [ ]:
data = {"x": [], "y": [], "cluster": []}
for i in range(len(assignment_values)):
  data["x"].append(vector_values[i][0])
  data["y"].append(vector_values[i][1])
  data["cluster"].append(assignment_values[i])
df = pd.DataFrame(data)
sns.lmplot("x", "y", data=df, 
           fit_reg=False, size=7, 
           hue="cluster", legend=False)
plt.show()

https://gist.github.com/narphorium/d06b7ed234287e319f18 tensorflow1.0~対応に書き換え済み